In [9]:
from astropy.table import Table
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import time
from sklearn import neighbors
from vast.voidfinder.multizmask import generate_mask
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker
import pickle

### Read in data and Generate Mask

In [2]:
file_name1 = "desi_slice_holes.fits"
data_table_V1 = Table.read(file_name1)

file_name2 = "desi_slice_maximals.fits"
data_table_V1max = Table.read(file_name2)

file_name3 = "sdss_slice_holes.fits"
data_table_V2 = Table.read(file_name3)

file_name4 = "sdss_slice_maximals.fits"
data_table_V2max = Table.read(file_name4)

In [12]:
galaxy_data_table = Table.read('sdss_slice_galaxies.fits')

mask, mask_resolution = generate_mask(galaxy_data_table, 
                                      .114, 
                                      smooth_mask=True,
                                      #h=h,
                                      )

Generating mask


### Functions that assist in the volume calculation

In [21]:
def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
    
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from first catalog.
    void_cat_B : astropy.Table
        Table of void data from second catalog.
        
    Returns
    -------
    x_min : float
    x_max : float
    y_min : float
    y_max : float
    z_min : float
    z_max : float
    """
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
    
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))

    return x_min, x_max, y_min, y_max, z_min, z_max

def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
    
    Returns
    -------
    xyz : list
        2D list of points in 3D space.
    """
    
    x_range = np.arange(x_min, x_max) # default spacing: 1 Megaparsec
    y_range = np.arange(y_min, y_max)
    z_range = np.arange(z_min, z_max)

    # Creating a meshgrid from the ranges to 
    X,Y,Z = np.meshgrid(x_range,y_range,z_range)

    x_points = np.ravel(X)
    y_points = np.ravel(Y)
    z_points = np.ravel(Z)
    
    point_coords = np.array([x_points, y_points, z_points])
    
    return point_coords

### Begin Volume Calculation

In [22]:
#This line creates the boundaries (like the whole min and max)
xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(data_table_V1, data_table_V2)

#This line makes creates the points in between 
pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

b = pts.shape
print(b)

(3, 1232616)


In [23]:
start_time = time.time()

rmin = 0
rmax = 312.89816 # Where are these computed?

points_boolean = np.ones(pts.shape[1], dtype = bool)

# print(time.time() - start_time)

# Declares MaskChecker object from VAST

mask_checker = MaskChecker(0,
                        mask,
                        mask_resolution,
                        rmin,
                        rmax,)

start_time = time.time()
for i in range(pts.shape[1]):

    # Current point will iterate through all points at 
    
    curr_pt = pts[:,i].reshape((1,3))
    
    # References MC object to declare if not_in_mask
    not_in_mask = mask_checker.not_in_mask(curr_pt[0])
#     print(not_in_mask, not bool(not_in_mask))

    points_boolean[i] = not bool(not_in_mask)
                                    
points_in_mask = pts[:,points_boolean]
(var, n_points) = points_in_mask.shape
print(time.time() - start_time)
print('\nPoints in Mask Shape:', points_in_mask.shape)

print('\nSum of Points IN:', np.sum(points_boolean))

print('\nSum of Points OUT:', np.sum(~points_boolean))

print('\nBoolean Shape:', points_boolean.shape)

print('\nPoints in Mask:',points_in_mask)


2.256254196166992

Points in Mask Shape: (3, 478584)

Sum of Points IN: 478584

Sum of Points OUT: 754032

Boolean Shape: (1232616,)

Points in Mask: [[-175.05090492 -175.05090492 -175.05090492 ...  -89.05090492
   -89.05090492  -89.05090492]
 [-259.28190302 -259.28190302 -259.28190302 ...  -41.28190302
   -41.28190302  -41.28190302]
 [  -4.65119965   -3.65119965   -2.65119965 ...    2.34880035
     3.34880035    4.34880035]]


#### KDTree Data Structure: Find closest point given large number of points
- Splits by dimension in halves from middle point
- Stores points as nodes in tree
- Continous to split within halves from middle point and stores childs
- Leafs nodes when splitting can occur no more

- Traverse through tree by dimensions. Levels: X -> Y -> Z -> X...
- Moves to child with nearest coordinate
- Repeats until reaches leaf node, return min dist.
- Travel back up tree and compare with nodes to determine no other possible min dist
- Prunes children with parents > min dist
Complexity: O(d*nlog*n) & O(dn) storage, d - dimensions




In [24]:
def kd_tree(void_cat):
    """We are creating a function to make a KDTree to find the number of points in 
    and out of a catalogue.
    
    Parameters
    ----------
    point_coords: ndarray has a shape of (3,N)
        This is the list of points to query the given void catalogue. N is the number of points given. 
    void_cat: Astropy Table
        This is the given void catalogue.
    
    Returns
    -------
    true_inside: ndarray of shape (N,1)
        Is this the boolean array of length N (same length as point_coords). True means that 1 point 
        is inside the hole.
    """
#############
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    sphere_coords = np.array([cx, cy, cz])

    #The .T is meant to transpose the array from (3,1054) to (1054,3)
    sphere_tree = neighbors.KDTree(sphere_coords.T)
    print("KDTree")

##############
    
    return sphere_tree

### Points Query
 - Parameters: Points in Mask Coor, KDTree, Data Catalog
 - Will get the distance and index of that point from KDTree
 - dist: Distance of point in KDtree from hole
 - idx: Index of point
 - Compare dist to the index of that point to the actual distance in the catalog
 - if < then the point falls within the hole
 - Return query of every point inside catalog

In [25]:
def point_query(point_coords, sphere_tree, void_cat):

    dist, idx = sphere_tree.query(point_coords.T, k = 1)

    true_inside = dist < void_cat['radius'][idx]
    
    print("Point Query")

    return true_inside

### Comparison of Void Catalogs
1. Declare False boolean arrays, size U = 21. 
    - Why? 21 is the sweet spot
2. Copy points_in_mask data matrix
3. Create KDTree of each Void Catalog
4. For loop through size of each array
    - Δ = Array size 3 with Random values. 
        - Why? In order for the KDTree to structure efficently,
          it must be randomized. Hence why we shift by 3.
    - Copy each array from points_in_mask and add Δ[i]
    - Run Point Querery Function on both data sets.
    - Gather sum of inside and sum of outside point for each
    - Logic functions
    
    
    

In [26]:
start_time = time.time()

U = 1

count_in_V1 = np.zeros(U)
count_out_V1 = np.zeros(U)

count_in_V2 = np.zeros(U)
count_out_V2 = np.zeros(U)

inside_both = np.zeros(U)
inside_neither = np.zeros(U)
inside_V1 = np.zeros(U)
inside_V2 = np.zeros(U)

points_in_mask_copy = points_in_mask.copy()

kdTree_V1 = kd_tree(data_table_V1)
kdTree_V2 = kd_tree(data_table_V2)


for i in range(U):
    

    
    delta = np.random.rand(3)
    
    points_in_mask_copy[0] = points_in_mask[0] + delta[0]
    points_in_mask_copy[1] = points_in_mask[1] + delta[1]
    points_in_mask_copy[2] = points_in_mask[2] + delta[2]
    
#     print(points_in_mask_copy[:,:10])
    
    true_inside_V1 = point_query(points_in_mask_copy, kdTree_V1, data_table_V1)

    count_in_V1[i] = np.sum(true_inside_V1)

    # The "~" inverts the array. So we have true_inside inverted to add up the falses instead of the trues
    count_out_V1[i] = np.sum(~true_inside_V1)
    
    # print(time.time() - start_time)

    # total_KIAS = count_in_KIAS + count_out_KIAS

    true_inside_V2 = point_query(points_in_mask_copy, kdTree_V2, data_table_V2)

    count_in_V2[i] = np.sum(true_inside_V2)

    # The "~" inverts the array. So we have true_inside inverted to add up the falses instead of the trues
    count_out_V2[i] = np.sum(~true_inside_V2)

    #total_DEL = count_in_DEL + count_out_DEL
    
    # This is the number of points that are inside both A and B
    inside_V1_and_V2 = np.logical_and(true_inside_V1, true_inside_V2)
    inside_both[i] = np.sum(inside_V1_and_V2)
    
    # This is the number of points that are in neither A and B
    not_inside_V1_and_V2 = np.logical_and(~true_inside_V1, ~true_inside_V2)
    inside_neither[i] = np.sum(not_inside_V1_and_V2)
    
    # This is the number of points that are in A but not B
    inside_v1 = np.logical_and(true_inside_V1, ~true_inside_V2)
    inside_V1[i] = np.sum(inside_v1)
    
    # This is the number of points that are not in A but are in B
    inside_v2 = np.logical_and(~true_inside_V1, true_inside_V2)
    inside_V2[i] = np.sum(inside_v2)
    
print(time.time() - start_time)
print('\nNumber of points inside V1:', count_in_V1)
print('\nNumber of points outside V2:', count_out_V1)
print('\nNumber of points inside V1:', count_in_V2)
print('\nNumber of points outside V2:', count_out_V2)
print("\nThis is the total number of points: {}".format(n_points))
# print("\nThis is the total number of points in Delaunay: {}".format(total_DEL))

KDTree
KDTree
Point Query
Point Query
1.3022019863128662

Number of points inside V1: [230926.]

Number of points outside V2: [247658.]

Number of points inside V1: [258640.]

Number of points outside V2: [219944.]

This is the total number of points: 478584


### This is the number of points that are inside both A and B

In [27]:
inside_V1_and_V2 = np.logical_and(true_inside_V1, true_inside_V2)

np.sum(inside_V1_and_V2), count_in_V1, count_in_V2

(197566, array([230926.]), array([258640.]))

### This is the number of points that are in neither A and B

In [28]:
not_inside_V1_and_V2 = np.logical_and(~true_inside_V1, ~true_inside_V2)

np.sum(not_inside_V1_and_V2), count_in_V1, count_in_V2

(186584, array([230926.]), array([258640.]))

### This is the number of points that are in A but not B

In [29]:
inside_V1 = np.logical_and(true_inside_V1, ~true_inside_V2)

np.sum(inside_V1)

33360

### This is the number of points that are not in A but are in B

In [30]:
inside_V2 = np.logical_and(~true_inside_V1, true_inside_V2)

np.sum(inside_V2)

61074

## Fraction of points: Calculate the ratios of the given sums

- in A
- in B
- in A ∧ B
- in ¬(A ∧ B)
- in A ∧¬B
- in B ∧¬A
-   |
-   |
- \ /
- The closer they are to zero, the more similar they are

Fraction of points in DESI voids

In [31]:
r_V1 = count_in_V1 / n_points
print(r_V1)

[0.48251927]


Fraction of points in SDSS voids

In [32]:
r_V2 = count_in_V2 / n_points
print(r_V2)

[0.54042759]


Fraction of points in both DESI and SDSS voids

In [34]:
r_V1_V2 = np.sum(inside_V1_and_V2) / n_points
print(r_V1_V2)

0.41281363355231265


Fraction of points in neither DESI nor SDSS voids

In [35]:
r_not_V1_V2 = np.sum(not_inside_V1_and_V2) / n_points
print(r_not_V1_V2)

0.389866773648931


Fraction of points in DESI but not SDSS voids

In [36]:
r_V1_not_V2 = np.sum(inside_V1) / n_points
print(r_V1_not_V2)

0.06970563161325911


Fraction of points in SDSS but not DESI voids

In [38]:
r_V2_not_V1 = np.sum(inside_V2) / n_points
print(r_V2_not_V1)

0.12761396118549723


In [47]:
#Sum = r_V1 + r_not_V1_V2 + r_V2_not_V1  
#Sum

### x̄ & σ of Void Catalogs Ratios

### Catalog 1

In [40]:
average_V1 = np.mean(count_in_V1)
r_average_V1 = average_V1 / n_points
std_V1 = np.std(count_in_V1)
r_std_V1 = std_V1 / n_points

print('\nRatio of V1 Points:', r_average_V1)
print('\nRatio SD:', r_std_V1)


Ratio of V1 Points: 0.48251926516557175

Ratio SD: 0.0


### Catalog 2

In [41]:
average_V2 = np.mean(count_in_V2)
r_average_V2 = average_V2 / n_points
std_V2 = np.std(count_in_V2)
r_std_V2 = std_V2 / n_points

print('\nRatio of V2 Points:', r_average_V2)
print('\nRatio SD:', r_std_V2)


Ratio of V2 Points: 0.5404275947378099

Ratio SD: 0.0


### x̄ & σ

In [42]:
average_inside = np.mean(inside_both)
r_average_inside = average_inside / n_points

std_both = np.std(inside_both)
r_std_both = std_both / n_points

print('\nRatio of Points Inside:', r_average_inside)
print('\nRatio SD:', r_std_both)


Ratio of Points Inside: 0.41281363355231265

Ratio SD: 0.0


In [43]:
average_outside = np.mean(inside_neither)
r_average_outside = average_outside / n_points

std_outside = np.std(inside_neither)
r_std_outside = std_outside / n_points

print('\nRatio of Points Outside:', r_average_outside)
print('\nRatio SD:', r_std_outside)


Ratio of Points Outside: 0.389866773648931

Ratio SD: 0.0


In [44]:
average_in_V1 = np.mean(inside_V1)
r_average_in_V1 = average_in_V1 / n_points

std_in_V1 = np.std(inside_neither)
r_std_in_V1 = std_in_V1 / n_points


print('\Ratio of Points in V1:', r_average_in_V1)
print('\nRatio SD:', r_std_in_V1)

\Ratio of Points in V1: 1.45649732572044e-07

Ratio SD: 0.0


In [45]:
average_in_V2 = np.mean(inside_V2)
r_average_in_V2 = average_in_V2 / n_points

std_in_V2 = np.std(inside_neither)
r_std_in_V2 = std_in_V2 / n_points

print('\Ratio of Points in V2:', average_in_V2)
print('\nRatio SD:', r_std_in_V2)

\Ratio of Points in V2: 0.12761396118549723

Ratio SD: 0.0
